In [1]:
# !capture --no-stderr
# !pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python 

In [2]:
# !pip install "nomic[local]"

In [1]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("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")
_set_env("nk-p4RbLXYiBQInfAQyrlCssat_n9w-697uXrq4dlCmq0o")

**Libraries**

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_nomic.embeddings import NomicEmbeddings

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain import hub
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

from langgraph.graph import END, StateGraph, START


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from scraper import *

In [ ]:
res = search_scholar("LLM")

In [ ]:
urls = []
for val in res.values():
    urls.append(val)

In [ ]:
urls[0]

'https://arxiv.org/pdf/2309.05519.pdf?trk=article-ssr-frontend-pulse_x-social-details_comments-action_comment-text'

In [ ]:
keys = res.keys()

In [ ]:
type(keys)

dict_keys

In [3]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    # "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    # "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

In [3]:
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define the paths to your local files
local_files = [
    "/mnt/Main Drive/Codes/LLM Agents/2309.05519v3.pdf",
]

# Class to wrap the file content
class Document:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata or {}

# Function to load the content from local PDF files
def load_local_pdf(file_path):
    doc = fitz.open(file_path)
    content = ""
    for page in doc:
        content += page.get_text()
    return Document(content, metadata={"source": file_path})

# Load the documents from the local files
docs = [load_local_pdf(file_path) for file_path in local_files]

# Split the documents using the text splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs)

In [4]:
doc_splits

[Document(metadata={'source': '/mnt/Main Drive/Codes/LLM Agents/2309.05519v3.pdf'}, page_content='NExT-GPT: Any-to-Any Multimodal LLM\nShengqiong Wu 1 Hao Fei 1 Leigang Qu 1 Wei Ji 1 Tat-Seng Chua 1\nAbstract\nWhile recently Multimodal Large Language Mod-\nels (MM-LLMs) have made exciting strides, they\nmostly fall prey to the limitation of only input-\nside multimodal understanding, without the abil-\nity to produce content in multiple modalities. As\nwe humans always perceive the world and com-\nmunicate with people through various modalities,\ndeveloping any-to-any MM-LLMs capable of ac-\ncepting and delivering content in any modality\nbecomes essential to human-level AI. To fill the\ngap, we present an end-to-end general-purpose\nany-to-any MM-LLM system, NExT-GPT. We\nconnect an LLM with multimodal adaptors and\ndifferent diffusion decoders, enabling NExT-GPT\nto perceive inputs and generate outputs in arbi-\ntrary combinations of text, image, video, and\naudio. By leveraging the 

**VectorDB**

In [5]:
# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
)
retriever = vectorstore.as_retriever()

Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


**Retrieval Grader**

In [6]:
local_llm = "llama3.1"

In [7]:
llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n
    If the document contains keywords related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.""",
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "llm"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

/tmp/ipykernel_605056/1271020123.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=local_llm, format="json", temperature=0)


{'score': 'yes'}


In [8]:
docs

[Document(metadata={'source': '/mnt/Main Drive/Codes/LLM Agents/2309.05519v3.pdf'}, page_content='Name\nParam\nName\nParam\nText\n—\n—\n—\n—\n—\n—\n—\n—\nImage\nVicuna\n7B\nTransformer\n31M\nSD\n1.3B\nAudio\n(LoRA\n33M\n)\nTransformer\n31M\nAudioLDM\n975M\nVideo\nImageBind\n1.2B\nGrouping\n28M\nTransformer\n32M\nZeroscope\n1.8B\nrepresentations are mapped into language-like representa-\ntions that are comprehensible to the LLM.\nLLM Understanding and Reasoning Stage\nAn LLM\nis used as the core agent of NExT-GPT. Technically, we\nemploy the Vicuna (7B-v0) (Chiang et al., 2023), which is\nthe open-source text-based LLM that is widely used in the\nexisting MM-LLMs (Su et al., 2023; Zhang et al., 2023c).\nLLM takes as input the representations from different modal-\nities and carries out semantic understanding and reasoning\nover the inputs. It outputs: 1) the textual responses directly,\nand 2) signal tokens of each modality that serve as instruc-\ntions to dictate the decoding layers on

**Generate**

In [9]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")
prompt = ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])


# LLM
llm = ChatOllama(model=local_llm, temperature=0)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

/home/neutrino/miniconda3/envs/Ml/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


It appears that you've provided a snippet from a research paper on the NExT-GPT system, which is an any-to-any multimodal large language model. The text doesn't seem to have a specific question or problem that needs to be answered, but rather provides information about the NExT-GPT framework and its architecture.

If you'd like, I can try to summarize the key points from the provided text:

* The NExT-GPT system is an any-to-any multimodal large language model that can understand and generate content in various modalities (e.g., text, image, video, audio).
* Most existing multimodal large language models (MM-LLMs) are limited to perceiving multimodal data without generating content in arbitrary modalities.
* The NExT-GPT system takes the mutual benefits of both MM-LLMs and LLMs as decision-makers, enabling end-to-end learning for any-to-any multimodal tasks.

Please let me know if you have a specific question or problem related to this text!


**Hallucination Grader**

In [10]:
llm = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template="""You are a grader assessing whether an answer is grounded in / supported by a set of facts. \n 
    Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    Here is the answer: {generation}
    Give a binary score 'yes' or 'no' score to indicate whether the answer is grounded in / supported by a set of facts. \n
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'no'}

**Answer Grader**

In [11]:
llm = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template="""You are a grader assessing whether an answer is useful to resolve a question. \n 
    Here is the answer:
    \n ------- \n
    {generation} 
    \n ------- \n
    Here is the question: {question}
    Give a binary score 'yes' or 'no' to indicate whether the answer is useful to resolve a question. \n
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
    input_variables=["generation", "question"],
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question, "generation": generation})

{'score': 'no'}

**Question Re-writer**

In [12]:
llm = ChatOllama(model=local_llm, temperature=0)

# Prompt
re_write_prompt = PromptTemplate(
    template="""You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the initial and formulate an improved question. \n
     Here is the initial question: \n\n {question}. Improved question with no preamble: \n """,
    input_variables=["generation", "question"],
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'It seems like you want me to rewrite the initial question into a more optimized form for vector store retrieval, but the initial question itself isn\'t provided.\n\nHowever, I can guide you on how to improve a question for better vector store retrieval:\n\n1. **Remove Preamble**: If your question starts with phrases like "I was wondering," "Can someone help me understand," or similar, remove them as they don\'t add value to the query and might reduce its effectiveness in search.\n\n2. **Focus on Key Entities**: Identify key entities (people, places, things) that are crucial for understanding the question. These should be included in your revised question.\n\n3. **Use Specific Keywords**: Incorporate specific keywords related to the topic or subject of interest. This can help in retrieving more relevant information from a vector store.\n\n4. **Simplify and Clarify**: Simplify the language used while maintaining clarity. Avoid ambiguity by making sure what you\'re asking is clear.\n\n5.

# Main Workflow

In [13]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

In [14]:
### Nodes

def retrieve(state):
    # Retrieve documents

    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}


def generate(state):
    # Generate answer

    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):

    # Determines whether the retrieved documents are relevant to the question.


    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score["score"]
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    # Transform the query to produce a better question.


    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}




def decide_to_generate(state):
    # Determines whether to generate an answer, or re-generate a question.


    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:

        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    # Determines whether the generation is grounded in the document and answers question.

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score["score"]

    # Check hallucination
    
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score["score"]
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [15]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

In [16]:
from pprint import pprint

# Run
inputs = {"question": "Explain the content thoroughly"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        print(f"Node '{key}':")
        pprint(value)
        print("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE---
Node 'retrieve':
{'documents': [Document(metadata={'source': '/mnt/Main Drive/Codes/LLM Agents/2309.05519v3.pdf'}, page_content='quality of base models, may produce low-quality or halluci-\nnated content that could be harmful. Users are cautioned to\ninterpret results carefully and adhere to licensing rules, with\ncommercial use prohibited. We prioritize data privacy by\nfollowing social media platform terms and obtaining user\nconsent when necessary, ensuring all personal information\nis anonymized or obfuscated. Additionally, we are vigilant\nin minimizing bias in dataset collection, striving for a repre-\nsentative and fair dataset that does not favor or disfavor any\nparticular group or perspective.\n9\nNExT-GPT: Any-to-Any Multimodal LLM\nReferences\nAgrawal, H., Anderson, P., Desai, K., Wang, Y., Chen, X.,\nJain, R., Johnson, M., Batra, D., Parikh, D., and Lee, S.\nnocaps: novel object captioning at scale. In Proceedings\nof the ICCV, pp. 8947–8956, 2019.\nAlayrac,